In [1]:
import pickle
# import graphvite as gv
import dataset
import nltk
import re
import spacy
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')



%reload_ext autoreload
%autoreload

ModuleNotFoundError: No module named 'nltk'

In [ ]:
file = open("simple_wikidata5m.pkl", "rb")
model = pickle.load(file)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings

In [ ]:
alias2entity = dataset.wikidata5m.load_alias("./entity.txt.gz")

In [ ]:
print(entity_embeddings[entity2id[alias2entity["invented"]]])

[ 2.60482699e-01 -2.64108598e-01  1.32692650e-01 -1.80898160e-01
  2.04402462e-01  3.02922130e-01  2.29415789e-01 -2.65274495e-01
  1.74571738e-01  5.07495478e-02 -1.53140053e-01  1.63881034e-01
  3.80817354e-01 -2.47925028e-01  1.42434612e-01  4.61547896e-02
  2.26035491e-01  3.26893598e-01 -3.08777392e-01  9.04703289e-02
 -3.79401118e-01  5.44060528e-01  2.40935609e-01  4.73862737e-01
  1.71523407e-01  2.77059644e-01  1.82027116e-01  6.17173493e-01
 -4.27625000e-01  3.13107938e-01 -3.16630810e-01 -4.91333276e-01
  2.56019652e-01 -2.33069897e-01 -2.12509558e-01  2.00872600e-01
 -4.32604164e-01  3.78530830e-01  2.59757489e-01  4.55503315e-01
  1.19828008e-01  2.73469687e-01 -3.76534820e-01  3.35429490e-01
 -3.16824853e-01 -2.15407729e-01  9.52157602e-02 -1.56584769e-01
 -2.01313972e-01  6.93463981e-02  3.10220689e-01  4.62196469e-01
  1.60467044e-01  1.08755626e-01  3.84727716e-01 -3.15181702e-01
  2.19031915e-01  1.48679212e-01 -4.16427493e-01  6.39831740e-03
 -3.80867481e-01 -2.42932

In [ ]:
nlp = spacy.load('en_core_web_sm') 
sentence = "Who invented machine learning, was it Steve Jobs"
doc = nlp(sentence) 
print(doc)
for ent in doc.ents: 
    print(ent.text, ent.label_) 

Who invented machine learning, was it Steve Jobs
Steve Jobs PERSON


In [ ]:
def clean_text(text):
    # Extract named entities
    nlp = spacy.load('en_core_web_sm') 
    named_entities = nlp(text).ents

    # Tokenize the text
    words = word_tokenize(text)

    # Remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]

    # Part-of-speech tagging
    tagged_words = pos_tag(words)

    cleaned_text = ' '.join(words).lower()
    
    return cleaned_text, named_entities

# Example usage:
text = "Who invented machine learning, was it Steve Jobs?"
cleaned_text, named_entities = clean_text(text)
print("Cleaned Text:", cleaned_text)
print("Named Entities:", named_entities)

Cleaned Text: invented machine learning steve jobs
Named Entities: (Steve Jobs,)


In [ ]:
def extract_continuous_word_sets(text, window_size):
    words = text.split()
    word_sets = []
    
    for i in range(len(words) - window_size + 1):
        word_set = ' '.join(words[i:i + window_size])
        word_sets.append(word_set)

    return word_sets

In [ ]:
def query_to_kg_embeddings(query):
    wiki_embeddings = {}
    cleaned_text, named_entities = clean_text(query)
    for ne in named_entities: 
        try:
            ne = ne.orth_.lower()
            wiki_embeddings[ne] = entity_embeddings[entity2id[alias2entity[ne]]]
            cleaned_text = cleaned_text.replace(ne, "")
        except Exception as e:
            print("KeyError: ", e)
    num_of_words = len(word_tokenize(cleaned_text))
    for i in range(num_of_words, 0, -1):
        word_sets = extract_continuous_word_sets(cleaned_text, i)
        for word in word_sets:
            try:
                wiki_embeddings[word] = entity_embeddings[entity2id[alias2entity[word]]]
                cleaned_text = cleaned_text.replace(word, "")
            except Exception as e:
                print("KeyError: ", e)

    return wiki_embeddings

In [ ]:
wiki_embeddings = (query_to_kg_embeddings("Who invented machine learning, was it Isaac Newton?"))

KeyError:  'invented machine learning'
KeyError:  'invented machine'


In [ ]:
for k in wiki_embeddings:
    print(k, len(wiki_embeddings[k]))

isaac newton 512
machine learning 512
invented 512
